
# Research Questions
1. How much duplication was there among programming for Syrian refugees in Jordan?
2. What fraction of aid inflows to Jordan related to refugee support and what fraction to development?
3. Biggest shift in donors? Donors who decreased their donations the most?

In [4]:
from lib.iati_datastore_utils import *
from lib.util_file import *
from google import genai
from google.genai import types
from definitions import GEMINI_API_KEY


In [5]:
jo_act = read_json("./data/iati/jordan_activities_w_transactions.json")

In [10]:
no_sector = []
for act in jo_act:
    if not act.get("sector_code") and not act.get("sector_narrative"):
        no_sector.append(act)

len(no_sector)

3587

# Identify Syrian Refugee Related Programs
Goal: Build a list of IATI identifier codes for programs at-least partially targeting Syrian refugees.

## Strategies
* Multi-lingual regular expressions - match keywords like "Syria," "Syrian Refugees," "refugees from Syria" acros multiple languages.
* LLM assisted description/title narrative scores.

## Obstacles
* Programs may assist different refugee groups
* Need to exclude programs assisting only Palestinian refugees, or Palestinian and Iraqi refugees, etc.

## Niceties
* Distinguish between programming in refugee camps (Za'atari, Azraq) and urban settings.
* Extrapolate sector where sector code is not present

## Fields to Add
* reg_exp_syria_ref - 0 or 1 if there's a regular expression match
* reg_exp_syria_camp - 0 or 1 if there's a regular expression match for Za'atari, Azraq, etc.
* reg_exp_usaid - 0 or 1 if there's a reg exp match for USAID
* llm_ref_group - ENUM - ["Syria", "Palestine", "Iraq", "Yemen", "Sudan", "Other"]
* llm_ref_setting - ENUM - ["camp", "urban"]
* llm_sector - str - llm's guess at the IATI sector codelist based on description
* llm_nexus - ENUM - ["humanitarian", "development", "both"]
* llm_orgs - List of international aid organizations involved in project.

In [1]:
# all_narratives = []
# for act in jo_act:
#     act_narratives = {k:v for k,v in act.items() if "narrative" in k}
#     act_narratives['iati_identifier'] = act['iati_identifier']
#     all_narratives.append(act_narratives)
#
# with open("./data/iati/narratives.txt", "w") as f:
#     for narr in all_narratives:
#         _id = narr.pop('iati_identifier')
#         _title = narr.pop("title_narrative")[0]
#         sep = ("\n"*2) + ("=" * 20) + "\n"
#         text = sep.join([k + "\n" + '\n'.join(v) for k,v in narr.items()])
#         f.write(_id + ": " + _title + sep + text + (sep*2))

In [32]:
def find_by_id(iati_id: str) -> Dict:
    matches = []
    for act in jo_act:
        if act['iati_identifier'] == iati_id:
            matches.append(act)
    return matches

In [33]:
find_by_id('KR-GOV-030-1342000_2024_018')

[{'sector_code': ['11220', '11240', '11130', '11182'],
  'iati_identifier': 'KR-GOV-030-1342000_2024_018',
  'title_narrative': ['Bridge Programme Phase 2 (Jordan) : Strengthening literacy skills and accessibility in education of children in Jordan'],
  'sector_narrative': ['Primary education',
   'Early childhood education',
   'Teacher training',
   'Educational research'],
  'transaction_value': [771000000.0, 771000000.0, 78000000.0],
  'activity_status_code': '2',
  'description_narrative': ["Expanding a reading program for children nationwide in Jordan, training experts in children's literacy education, and hosting forums to promote literacy education among children."],
  'transaction_value_currency': ['KRW', 'KRW', 'KRW'],
  'transaction_provider_org_ref': ['KR-GOV-030', 'KR-GOV-030', 'KR-GOV-030'],
  'transaction_value_value_date': ['2025-01-01T00:00:00Z',
   '2025-01-01T00:00:00Z',
   '2025-03-31T00:00:00Z'],
  'transaction_provider_org_type': ['10', '10', '10'],
  'transaction

In [3]:
client = genai.Client(api_key=GEMINI_API_KEY)
client

In [4]:
response = client.models.generate_content(
    model='gemini-2.0-flash-001', contents='Why is the sky blue?'
)
print(response.text)

The sky is blue due to a phenomenon called **Rayleigh scattering**. Here's a breakdown of why:

*   **Sunlight and its Colors:** Sunlight appears white to our eyes, but it's actually composed of all the colors of the rainbow (red, orange, yellow, green, blue, indigo, violet).

*   **Entering the Atmosphere:** When sunlight enters the Earth's atmosphere, it collides with air molecules (mostly nitrogen and oxygen).

*   **Scattering:** This collision causes the light to scatter in different directions.

*   **Rayleigh Scattering and Wavelength:** The amount of scattering depends on the wavelength of the light. Shorter wavelengths (blue and violet light) are scattered much more strongly than longer wavelengths (red and orange light). This is Rayleigh scattering in action, which is inversely proportional to the fourth power of the wavelength.

*   **Why Blue Dominates:** Violet light is scattered even more than blue light, but our eyes are more sensitive to blue light, and the sun emits sl

In [2]:
len(jo_act)

NameError: name 'jo_act' is not defined